In [381]:
import pandas
import xml.etree.ElementTree as et 
import dateutil.parser
import pytz, datetime
import json
steps_dict = {'OnwardPricedItinerary':'start','ReturnPricedItinerary':'finish','Pricing':'price'}
CONFIG_FLIGHT = \
    {
        'Carrier': {
            'dtype': 'text',
            'attr': {
                'id': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Airline_id'
                }
            },
            'descrip': 'Airline'
        },
        'FlightNumber': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Number_of_flight'
        },
        'Source': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'From'
        },
        'Destination': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'To'
        },
        'DepartureTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_from'
        },
        'ArrivalTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_to'
        },
        'Class': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Class'
        },
        'NumberOfStops': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Count_stops'
        },
        'FareBasis': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Trip_id'
        },
        'WarningText': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Description'
        },
        'TicketType': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Type_ticket'
        }

    }
CONFIG_PRICE = \
    {
        'ServiceCharges': {
            'dtype': 'float',
            'attr': {
                'type': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Type_seat'
                },
                'ChargeType': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Rate_type'
                }
            },
            'descrip': 'Charges'
        },
    }


In [7]:
tree = et.parse('xmls/RS_Via-3.xml')


In [110]:
root = tree.getroot()
for c in root:
root.attrib

RequestId
PricedItineraries


{'RequestTime': '28-09-2015 20:23:49', 'ResponseTime': '28-09-2015 20:23:56'}

In [164]:
# for flights in root.findall('.//Flights'):
#     p = flights

for child in root.findall('.//Flight'):
    print(list(child))

    carrier = child.find('Carrier').text if child is not None else None
    source = child.find('Source').text if child is not None else None
    n = child.find('FlightNumber').text if child is not None else None
    destination = child.find('Destination').text if child is not None else None
    print(source,'-->',destination,n,p)
        


#     carrier = child.find('Carrier').text if child is not None else None
#     print(carrier)


[<Element 'Carrier' at 0x113bae2c8>, <Element 'FlightNumber' at 0x113bae368>, <Element 'Source' at 0x113bae408>, <Element 'Destination' at 0x113bae4a8>, <Element 'DepartureTimeStamp' at 0x113bae548>, <Element 'ArrivalTimeStamp' at 0x113bae5e8>, <Element 'Class' at 0x113bae688>, <Element 'NumberOfStops' at 0x113bae728>, <Element 'FareBasis' at 0x113bae7c8>, <Element 'WarningText' at 0x113bae8b8>, <Element 'TicketType' at 0x113bae958>]
DXB --> DEL 996 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0x113baea48>, <Element 'FlightNumber' at 0x113baeae8>, <Element 'Source' at 0x113baeb88>, <Element 'Destination' at 0x113baec28>, <Element 'DepartureTimeStamp' at 0x113baecc8>, <Element 'ArrivalTimeStamp' at 0x113baed68>, <Element 'Class' at 0x113baee08>, <Element 'NumberOfStops' at 0x113baeea8>, <Element 'FareBasis' at 0x113baef48>, <Element 'WarningText' at 0x113bb1098>, <Element 'TicketType' at 0x113bb1138>]
DEL --> BKK 332 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0

DOH --> BKK 830 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0x113d83688>, <Element 'FlightNumber' at 0x113d83728>, <Element 'Source' at 0x113d837c8>, <Element 'Destination' at 0x113d83868>, <Element 'DepartureTimeStamp' at 0x113d83908>, <Element 'ArrivalTimeStamp' at 0x113d839a8>, <Element 'Class' at 0x113d83a48>, <Element 'NumberOfStops' at 0x113d83ae8>, <Element 'FareBasis' at 0x113d83b88>, <Element 'WarningText' at 0x113d83c78>, <Element 'TicketType' at 0x113d83d18>]
BKK --> DOH 833 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0x113d83e08>, <Element 'FlightNumber' at 0x113d83ea8>, <Element 'Source' at 0x113d83f48>, <Element 'Destination' at 0x113d86048>, <Element 'DepartureTimeStamp' at 0x113d860e8>, <Element 'ArrivalTimeStamp' at 0x113d86188>, <Element 'Class' at 0x113d86228>, <Element 'NumberOfStops' at 0x113d862c8>, <Element 'FareBasis' at 0x113d86368>, <Element 'WarningText' at 0x113d86458>, <Element 'TicketType' at 0x113d864f8>]
DOH --> DXB 1002 <Elemen

DOH --> DXB 1002 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0x113ee0a48>, <Element 'FlightNumber' at 0x113ee0ae8>, <Element 'Source' at 0x113ee0b88>, <Element 'Destination' at 0x113ee0c28>, <Element 'DepartureTimeStamp' at 0x113ee0cc8>, <Element 'ArrivalTimeStamp' at 0x113ee0d68>, <Element 'Class' at 0x113ee0e08>, <Element 'NumberOfStops' at 0x113ee0ea8>, <Element 'FareBasis' at 0x113ee0f48>, <Element 'WarningText' at 0x113ee2098>, <Element 'TicketType' at 0x113ee2138>]
DXB --> DOH 1065 <Element 'Flight' at 0x113fc5ae8>
[<Element 'Carrier' at 0x113ee2228>, <Element 'FlightNumber' at 0x113ee22c8>, <Element 'Source' at 0x113ee2368>, <Element 'Destination' at 0x113ee2408>, <Element 'DepartureTimeStamp' at 0x113ee24a8>, <Element 'ArrivalTimeStamp' at 0x113ee2548>, <Element 'Class' at 0x113ee25e8>, <Element 'NumberOfStops' at 0x113ee2688>, <Element 'FareBasis' at 0x113ee2728>, <Element 'WarningText' at 0x113ee2818>, <Element 'TicketType' at 0x113ee28b8>]
DOH --> BKK 830 <Eleme

In [178]:
flights = root.findall('.//Flights') 
for flight in flights:
    print(list(flight))

[<Element 'OnwardPricedItinerary' at 0x113bae188>, <Element 'ReturnPricedItinerary' at 0x113bb1188>, <Element 'Pricing' at 0x113bb4188>]
[<Element 'Flight' at 0x113bae228>, <Element 'Flight' at 0x113bae9a8>]
[<Element 'Flight' at 0x113bb1228>, <Element 'Flight' at 0x113bb19a8>]
[<Element 'OnwardPricedItinerary' at 0x113bb4318>, <Element 'ReturnPricedItinerary' at 0x113bb6318>, <Element 'Pricing' at 0x113bb8318>]
[<Element 'Flight' at 0x113bb43b8>, <Element 'Flight' at 0x113bb4b38>]
[<Element 'Flight' at 0x113bb63b8>, <Element 'Flight' at 0x113bb6b38>]
[<Element 'OnwardPricedItinerary' at 0x113bb84a8>, <Element 'ReturnPricedItinerary' at 0x113bbb4a8>, <Element 'Pricing' at 0x113bbf4a8>]
[<Element 'Flight' at 0x113bb8548>, <Element 'Flight' at 0x113bb8cc8>]
[<Element 'Flight' at 0x113bbb548>, <Element 'Flight' at 0x113bbbcc8>]
[<Element 'OnwardPricedItinerary' at 0x113bbf638>, <Element 'ReturnPricedItinerary' at 0x113bc1638>, <Element 'Pricing' at 0x113bc2638>]
[<Element 'Flight' at 0x11

In [186]:
def recur(root):
    if 'Flights' == root.tag:
        return [root]
    flights = list()
    for child in root:
        flights += recur(child)
    return flights

In [187]:
trips = recur(root)

In [246]:
for trip in trips:
    for step in list(trip):
        parse_step(step)

    

Carrier
<Element 'Carrier' at 0x113bae2c8>
<Element 'Carrier' at 0x113bae2c8> {'id': {'dtype': 'text', 'attr': False, 'descrip': 'Airline_id'}}
id
None


AttributeError: 'NoneType' object has no attribute 'tag'

In [204]:
def parse_step(node):
    tag = node.tag
    if tag in steps_dict:
        if steps_dict[tag] == 'start':
            get_start_flight(node)
        elif steps_dict[tag] == 'finish':
            pass
        elif steps_dict[tag] == 'price':
            pass
    return
    

In [226]:
def get_start_flight(node):
    for flight in node.findall('.//Flight'):
#         carrier = flight.find('Carrier').text if flight is not None else None
#         source = flight.find('Source').text if flight is not None else None
#         n = flight.find('FlightNumber').text if flight is not None else None
#         destination = flight.find('Destination').text if flight is not None else None
#         start_time = flight.find('DepartureTimeStamp').text if flight is not None else None
#         start_time = dateutil.parser.parse(start_time)
#         print(source,'-->',destination,n,start_time)
        get_tag(flight,CONFIG_FLIGHT)
    

In [245]:
def get_tag(root,config):
    for tag in config:
        print(tag)
        item = root.find(tag) if root is not None else None
        print(item)
        if config[tag]['attr']:
            print(item,config[tag]['attr'])
            get_tag(item,config[tag]['attr'])
        data = check_dtype(item,config)
        print(data)

In [237]:
def check_dtype(item,config):
    tag = item.tag
    if tag in config:
        data = item.text
        dtype = config[tag]['dtype']
        if dtype == 'datetime':
            data = dateutil.parser.parse(data)
        elif dtype == 'int':
            data = int(data)
        elif dtype == 'float':
            data = float(data)
    return data

In [453]:
import xml.etree.ElementTree as et
import dateutil.parser as parser
import pandas as pd
import datetime
from timezone import get_time_by_local


# tree = et.parse('xmls/RS_Via-3.xml')
# root = tree.getroot()


class Query:
    def __init__(self, xml):
        self.xml = xml

    def get_xml_root(self):
        tree = et.parse(self.xml)
        root = tree.getroot()
        return root


# class ParseTrips:
#     def __init__(self, root):
#         self.root = root
#     def get_trips(self):


first_q = Query('xmls/RS_ViaOW.xml')
root = first_q.get_xml_root()

steps_dict = {'OnwardPricedItinerary': 'start',
              'ReturnPricedItinerary': 'finish', 'Pricing': 'price'}

CONFIG_FLIGHT = \
    {
        'Carrier': {
            'dtype': 'text',
            'attr': {
                'id': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Airline_id'
                }
            },
            'descrip': 'Airline'
        },
        'FlightNumber': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Number_of_flight'
        },
        'Source': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'From'
        },
        'Destination': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'To'
        },
        'DepartureTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_from'
        },
        'ArrivalTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_to'
        },
        'Class': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Class'
        },
        'NumberOfStops': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Count_stops'
        },
        'FareBasis': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Trip_id'
        },
        'WarningText': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Description'
        },
        'TicketType': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Type_ticket'
        }

    }
CONFIG_PRICE = \
    {
        'ServiceCharges': {
            'dtype': 'float',
            'attr': {
                'type': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Person_type'
                },
                'ChargeType': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Rate_type'
                }
            },
            'descrip': 'Charges'
        },
    }


def parse_step(node):
    tag = node.tag
    l_step = list()
    if tag in steps_dict:
        l_step = parse_flights(node, tag)
    return l_step


def get_tag(root, config, attr=None):
    tags = dict()
    for tag in config:
        if attr:
            item = root.attrib
            if len(item) > 1:
                item = {k: item[k] for k in item if k == tag}
        else:
            item = root.find(tag) if root is not None else None
        if config[tag]['attr']:
            tags.update(get_tag(item, config[tag]['attr'], attr=True))
        data = check_dtype(item, config, attr)
        tags.update(data)
    return tags


def check_dtype(item, config, attr):
    d = dict()
    if attr is not None:
        tag = next(iter(item))
        data = item[tag]
    else:
        tag = item.tag
        data = item.text
    if tag in config:
        descrip = config[tag]['descrip']
        dtype = config[tag]['dtype']
        if dtype == 'datetime':
            data = parser.parse(data)
        elif dtype == 'int':
            data = int(data)
        elif dtype == 'float':
            data = float(data)
    d[descrip] = data
    return d


def parse_flights(node, tag):
    trip_flights = list()
    transfer_flight = None
    for cur_flight in range(len(node.findall('.//Flight')) - 1, -1, -1):
        row = get_tag(node.findall('.//Flight')[cur_flight], CONFIG_FLIGHT)
        row['Tag'] = tag
        from_local_t = get_time_by_local(row['From'], row['Time_from'])
        to_local_t = get_time_by_local(row['To'], row['Time_to'])
        if from_local_t['code'] == 200 and to_local_t['code'] == 200:
            row['Time_from_local'] = parser.parse(from_local_t['data'])
            row['Time_to_local'] = parser.parse(to_local_t['data'])
        if cur_flight > 0:
            row['transfer_to'] = transfer_flight
            transfer_flight = row['Number_of_flight']
        else:
            row['transfer_to'] = transfer_flight
        trip_flights.append(row)
    return trip_flights


def parse_trip_price(trip, trip_id):
    trip_prices = list()
    for price in trip.findall('.//Pricing'):
        for single_price in price.findall('ServiceCharges'):
            price_d = dict()
            price_d.update(check_dtype(single_price, CONFIG_PRICE, attr=None))
            price_d['currency'] = price.attrib['currency']
            price_d['Trip_id'] = trip_id
            price_attrib = single_price.attrib
            if price_attrib:
                for k in price_attrib:
                    item = {k: price_attrib[k]}
                    price_d.update(
                        check_dtype(item,
                                    CONFIG_PRICE[single_price.tag]['attr'],
                                    attr=True))
            trip_prices.append(price_d)
    return trip_prices


def get_trips(root):
    if 'Flights' == root.tag:
        return [root]
    flights = list()
    for child in root:
        flights += get_trips(child)
    return flights


t1 = datetime.datetime.now()
trips = get_trips(root)
flights = list()
prices = list()
for trip in trips:
    flight = list()
    for trips_step in list(trip):
        trips_step = parse_step(trips_step)
        if trips_step:
            flight.extend(trips_step)
    if flight:
        flights.extend(flight)
        trip_id = flight[0]['Trip_id']
    price = parse_trip_price(trip, trip_id)
    if price:
        prices.extend(price)
df = pd.DataFrame(data=flights)
prices = pd.DataFrame(data=prices)
print(datetime.datetime.now() - t1)


0:00:00.303253


In [454]:
df = df.sort_values(['Trip_id','Tag','transfer_to'])
df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
61,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,830.0
60,Qatar Airways,QR,N,0,None,DOH,830,OnwardPricedItinerary,2018-10-27 20:40:00,2018-10-27 20:40:00,2018-10-28 06:55:00,2018-10-28 02:55:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN
77,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0
76,Qatar Airways,QR,N,0,None,DOH,832,OnwardPricedItinerary,2018-10-28 08:30:00,2018-10-28 08:30:00,2018-10-28 18:45:00,2018-10-28 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN
194,Qatar Airways,QR,M,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,834.0
193,Qatar Airways,QR,M,0,None,DOH,834,OnwardPricedItinerary,2018-10-28 01:20:00,2018-10-28 01:20:00,2018-10-28 11:35:00,2018-10-28 07:35:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN
63,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,836.0
62,Qatar Airways,QR,N,0,None,DOH,836,OnwardPricedItinerary,2018-10-28 01:55:00,2018-10-28 01:55:00,2018-10-28 12:10:00,2018-10-28 08:10:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN
103,Qatar Airways,QR,S,0,None,DWC,1033,OnwardPricedItinerary,2018-10-27 06:25:00,2018-10-27 05:25:00,2018-10-27 06:30:00,2018-10-27 06:30:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0
102,Qatar Airways,QR,S,0,None,DOH,832,OnwardPricedItinerary,2018-10-27 08:30:00,2018-10-27 08:30:00,2018-10-27 18:45:00,2018-10-27 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN


In [455]:
df.dtypes

Airline                     object
Airline_id                  object
Class                       object
Count_stops                  int64
Description                 object
From                        object
Number_of_flight             int64
Tag                         object
Time_from           datetime64[ns]
Time_from_local     datetime64[ns]
Time_to             datetime64[ns]
Time_to_local       datetime64[ns]
To                          object
Trip_id                     object
Type_ticket                 object
transfer_to                float64
dtype: object

In [316]:
# БЕЗ ПЕРЕСАДОК
df.loc[(df['From'] == 'DXB') & (df['To'] == 'BKK'),'flag'] = 'without_transfer'
df



,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_to,To,Trip_id,Type_ticket,transfer_to,flag
459,Qatar Airways,QR,N,0,None,BKK,831,finish,2018-10-30 08:35:00,2018-10-30 11:35:00,DOH,\n 2820231f40c802-0...,E,1016.0,NaN
458,Qatar Airways,QR,N,0,None,DOH,1016,finish,2018-10-30 13:00:00,2018-10-30 15:10:00,DXB,\n 2820231f40c802-0...,E,NaN,NaN
457,Qatar Airways,QR,N,0,None,DWC,1031,start,2018-10-22 19:45:00,2018-10-22 19:50:00,DOH,\n 2820231f40c802-0...,E,830.0,NaN
456,Qatar Airways,QR,N,0,None,DOH,830,start,2018-10-22 21:00:00,2018-10-23 07:25:00,BKK,\n 2820231f40c802-0...,E,NaN,NaN
471,Qatar Airways,QR,N,0,None,BKK,833,finish,2018-10-30 20:45:00,2018-10-30 23:55:00,DOH,\n 2820231f40c802-0...,E,1032.0,NaN
470,Qatar Airways,QR,N,0,None,DOH,1032,finish,2018-10-31 01:35:00,2018-10-31 03:40:00,DWC,\n 2820231f40c802-0...,E,NaN,NaN
469,Qatar Airways,QR,N,0,None,DWC,1031,start,2018-10-22 19:45:00,2018-10-22 19:50:00,DOH,\n 2820231f40c802-0...,E,830.0,NaN
468,Qatar Airways,QR,N,0,None,DOH,830,start,2018-10-22 21:00:00,2018-10-23 07:25:00,BKK,\n 2820231f40c802-0...,E,NaN,NaN
463,Qatar Airways,QR,N,0,None,BKK,833,finish,2018-10-30 20:45:00,2018-10-30 23:55:00,DOH,\n 2820231f40c802-0...,E,1002.0,NaN
462,Qatar Airways,QR,N,0,None,DOH,1002,finish,2018-10-31 01:00:00,2018-10-31 03:10:00,DXB,\n 2820231f40c802-0...,E,NaN,NaN


In [461]:
# количество часов рейс

df['flight_time'] = df.apply(lambda row: row['Time_to_local'] - row['Time_from_local'],axis=1)

a = df.groupby(['Trip_id','Tag']).sum()


In [452]:
import io
import requests
import pandas as pd
import redis
import pytz
from tzlocal import get_localzone
from datetime import datetime

r = redis.Redis('localhost')


def search_timezone_by_iata(search_code):
    status = check_iata_spr()
    if status == 200:
        search_code = search_code.upper()
        timezone = r.hget('iata_timezone', search_code)
        if timezone is not None:
            return {'code': 200, 'data': timezone.decode('UTF-8')}
        return {'code': 500, 'msg': 'No data by key in redis'}
    return status


def get_iata_country_spr():
    url = "https://raw.githubusercontent.com/opentraveldata/opentraveldata/master/opentraveldata/optd_por_public.csv"
    try:
        s = requests.get(url).content
    except ConnectionError:
        return {'code': 500, 'msg': 'ConnectionError'}
    country_map = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='^')
    iata_dict = country_map.loc[:,
                ['iata_code', 'country_code', 'timezone']].drop_duplicates(
        subset=['iata_code']).to_dict(orient='records')
    iata_dict = {k['iata_code']: k['timezone'] for k in iata_dict}
    r.hmset('iata_timezone', iata_dict)
    check_r = r.exists('iata_timezone')
    if check_r:
        return {'code': 200}
    else:
        return {'code': 500, 'msg': 'No data in redis'}


def check_iata_spr():
    if not r.exists('iata_timezone'):
        load_spr = get_iata_country_spr()
        if 'code' in load_spr:
            if load_spr['code'] == 200:
                return 200
        return load_spr
    else:
        return 200


def get_time_by_local(iata_code, dt):
    timezone = search_timezone_by_iata(iata_code)
    if timezone['code'] == 200:
        timezone = pytz.timezone(timezone['data'])
        timezone_dt = timezone.localize(dt)
        local_zone = pytz.timezone(str(get_localzone()))
        local_dt = timezone_dt.astimezone(local_zone)
        local_dt = datetime.strftime(local_dt, '%Y-%m-%d %T')
        return {'code': 200, 'data': local_dt}
    return timezone


# t1 = datetime.now()
print(get_time_by_local('HKT', datetime(2002, 10, 27, 6, 0, 0)))
# print(datetime.now() - t1)


{'code': 200, 'data': '2002-10-27 02:00:00'}


In [438]:
df['Time_from_local'] = df.apply(lambda row:get_time_by_local(row['From'],row['Time_from']),axis=1)
df['Time_to_local'] = df.apply(lambda row:get_time_by_local(row['To'],row['Time_to']),axis=1)
# df['Time_from_local'] = df.apply(lambda row:1)

In [439]:
df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_to,To,Trip_id,Type_ticket,transfer_to,flag,flight_time,Time_from_local,Time_to_local
459,Qatar Airways,QR,N,0,None,BKK,831,finish,2018-10-30 08:35:00,2018-10-30 11:35:00,DOH,\n 2820231f40c802-0...,E,1016.0,NaN,03:00:00,"{'code': 200, 'data': 2018-10-30 04:35:00+03:00}","{'code': 200, 'data': 2018-10-30 11:35:00+03:00}"
458,Qatar Airways,QR,N,0,None,DOH,1016,finish,2018-10-30 13:00:00,2018-10-30 15:10:00,DXB,\n 2820231f40c802-0...,E,NaN,NaN,02:10:00,"{'code': 200, 'data': 2018-10-30 13:00:00+03:00}","{'code': 200, 'data': 2018-10-30 14:10:00+03:00}"
457,Qatar Airways,QR,N,0,None,DWC,1031,start,2018-10-22 19:45:00,2018-10-22 19:50:00,DOH,\n 2820231f40c802-0...,E,830.0,NaN,00:05:00,"{'code': 200, 'data': 2018-10-22 18:45:00+03:00}","{'code': 200, 'data': 2018-10-22 19:50:00+03:00}"
456,Qatar Airways,QR,N,0,None,DOH,830,start,2018-10-22 21:00:00,2018-10-23 07:25:00,BKK,\n 2820231f40c802-0...,E,NaN,NaN,10:25:00,"{'code': 200, 'data': 2018-10-22 21:00:00+03:00}","{'code': 200, 'data': 2018-10-23 03:25:00+03:00}"
471,Qatar Airways,QR,N,0,None,BKK,833,finish,2018-10-30 20:45:00,2018-10-30 23:55:00,DOH,\n 2820231f40c802-0...,E,1032.0,NaN,03:10:00,"{'code': 200, 'data': 2018-10-30 16:45:00+03:00}","{'code': 200, 'data': 2018-10-30 23:55:00+03:00}"
470,Qatar Airways,QR,N,0,None,DOH,1032,finish,2018-10-31 01:35:00,2018-10-31 03:40:00,DWC,\n 2820231f40c802-0...,E,NaN,NaN,02:05:00,"{'code': 200, 'data': 2018-10-31 01:35:00+03:00}","{'code': 200, 'data': 2018-10-31 02:40:00+03:00}"
469,Qatar Airways,QR,N,0,None,DWC,1031,start,2018-10-22 19:45:00,2018-10-22 19:50:00,DOH,\n 2820231f40c802-0...,E,830.0,NaN,00:05:00,"{'code': 200, 'data': 2018-10-22 18:45:00+03:00}","{'code': 200, 'data': 2018-10-22 19:50:00+03:00}"
468,Qatar Airways,QR,N,0,None,DOH,830,start,2018-10-22 21:00:00,2018-10-23 07:25:00,BKK,\n 2820231f40c802-0...,E,NaN,NaN,10:25:00,"{'code': 200, 'data': 2018-10-22 21:00:00+03:00}","{'code': 200, 'data': 2018-10-23 03:25:00+03:00}"
463,Qatar Airways,QR,N,0,None,BKK,833,finish,2018-10-30 20:45:00,2018-10-30 23:55:00,DOH,\n 2820231f40c802-0...,E,1002.0,NaN,03:10:00,"{'code': 200, 'data': 2018-10-30 16:45:00+03:00}","{'code': 200, 'data': 2018-10-30 23:55:00+03:00}"
462,Qatar Airways,QR,N,0,None,DOH,1002,finish,2018-10-31 01:00:00,2018-10-31 03:10:00,DXB,\n 2820231f40c802-0...,E,NaN,NaN,02:10:00,"{'code': 200, 'data': 2018-10-31 01:00:00+03:00}","{'code': 200, 'data': 2018-10-31 02:10:00+03:00}"
